In [1]:
import os
import sys
import argparse
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

from tqdm import tqdm

import compute_lcs

In [2]:
# load a dna file
path_lcs_mixed = '../../formatted_datasets/lcs/mixed1.csv'
path_dna_mixed = '../../formatted_datasets/dna/mixed1.csv'


# load in dataframes
lcs_mixed = pd.read_csv(path_lcs_mixed)
dna_mixed = pd.read_csv(path_dna_mixed)

In [4]:
# extraire 20 éléments du dataframe dna_mixed
dna_mixed = dna_mixed.sample(n=20, random_state=1)
lcs_mixed = compute_lcs.from_df(dna_mixed)
print(lcs_mixed)

LCS calculé en 4.485242128372192 secondes.
     k length                                               path
0    2    999  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
1    3    979  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
2    4    971  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
3    5    864  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
4    6    858  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
5    7    570  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
6    8    460  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
7    9    457  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
8   10    449  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
9   11    448  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
10  12    427  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
11  13    413  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
12  14     30                     AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
13  15     23                            AAAAAA

In [5]:
# max k in the lcs dataframe
max_k = lcs_mixed['k'].max()
print(f"Max k in lcs: {max_k}")

Max k in lcs: 20


In [26]:

kval = []
# itérer sur chaque ligne du dataframe dna_mixed et afficher la valeur de la colonne DNA
for index, row in tqdm(dna_mixed.iterrows(), total=dna_mixed.shape[0]):
    userdna=row['DNA']
    for index,row in lcs_mixed.iterrows():
        lcs=str(row['path'])
        lengthlcs = int(row['length'])
        if lcs in str(userdna):
            kval.append(row['length'])
            #print(f"Found (L:{lengthlcs}) {lcs} in {userdna}")
            break

print(dna_mixed.shape)
print(len(kval))

#dna_mixed['kval'] = kval
#print(dna_mixed)


        


100%|██████████| 36/36 [00:00<00:00, 1993.17it/s]

(36, 4)
20


In [119]:
# afficher la moyenne de la colonne kval pour label BOT et pour HUMAN
print(dna_mixed[dna_mixed['label'] == 'BOT']['kval'].mean())
print(dna_mixed[dna_mixed['label'] == 'HUMAN']['kval'].mean())

6.89606458123108
183.234106962664
2.0
2.0


In [120]:
# split dna into train dataframe and test dataframe (50/50)
X_train, X_test, y_train, y_test = train_test_split(dna_mixed.drop(columns=['label']), dna_mixed['label'], test_size=0.5, random_state=42)




In [138]:
acc_list = []

for k in range(2, max_k+1):
    y_pred = X_test['kval'] < k
    # change True into 'BOT' and False into 'HUMAN'
    y_pred = y_pred.apply(lambda x: 'BOT' if x else 'HUMAN')
    acc = accuracy_score(y_test, y_pred)
    acc_list.append(acc)
    



In [139]:
print(max(acc_list))

0.6427850655903128


In [76]:
# apprentissage en testant pour chaque valeur de k

k = 400

# créer un dataframe contenant tous les path du dataframe lcs de 2 à k


# pour chaque utilisateur du training set, vérifier si au moins un des lcs de lcs_train est dans son adn (colonne DNA)
# si oui, alors c'est un bot, sinon c'est un humain
# stocker les prédictions dans y_pred

lst_acc = []
for k in tqdm(range(3, max_k+1,30)):
    y_pred = []
    lcs_train = lcs_mixed[lcs_mixed['k'] <= k]['path'].tolist()
    print(lcs_train)
    print("Taille de lcs_train : ", len(lcs_train))
    for dna in X_train['DNA']:   
        for lcs in lcs_train:
            if lcs in dna:
                y_pred.append("BOT")
            else:
                y_pred.append("HUMAN")
            break
    # compter le nb de HUMAN et de BOT dans y_pred. afficher, avec la valeur de k.tout sur une ligne
    print(f"K : {k} - HUMAN : {y_pred.count('HUMAN')} - BOT : {y_pred.count('BOT')}")
    accuracy = accuracy_score(y_trainlst, y_pred)
    lst_acc.append(accuracy)

# trouver le k qui donne la meilleure accuracy
best_k = lst_acc.index(max(lst_acc)) + 2








  0%|          | 0/66 [00:00<?, ?it/s]

['AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

ValueError: max() arg is an empty sequence

In [72]:
print(lst_acc)

[0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514, 0.5368314833501514,

In [58]:
y_trainlst = y_train.tolist()

# calculer l'accuracy, le f1 et print
accuracy = accuracy_score(y_trainlst, y_pred)
f1 = f1_score(y_trainlst, y_pred, pos_label="HUMAN")
print(f"Accuracy: {accuracy}")
print(f"F1: {f1}")

Accuracy: 0.5368314833501514
F1: 0.6796929518492673
